In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import torchvision, torch

In [9]:
import sys, os
sys.path.insert(0,"../torchxrayvision/")
import torchxrayvision as xrv
import matplotlib.pyplot as plt
import torch
from torch.nn import functional as F
import glob
import numpy as np
import skimage, skimage.filters
%matplotlib inline
import captum, captum.attr
import torch, torch.nn
import pickle
import attribution
import pandas as pd
import datasets
import shutil
import pickle


In [10]:
ae = xrv.autoencoders.ResNetAE(weights="101-elastic").cuda()

In [11]:
models = [xrv.models.DenseNet(weights="all").cuda(),
          xrv.models.DenseNet(weights="mimic_ch").cuda(),
          xrv.baseline_models.jfhealthcare.DenseNet().cuda()]

In [13]:
for_eval = {
            "Cardiomegaly":datasets.get_data("nih"),
            'Mass':datasets.get_data("nih"),
            #'Nodule':datasets.get_data("nih"), 
            #"Atelectasis":datasets.get_data("nih"),
            #"Effusion":datasets.get_data("nih"),
            #"Lung Opacity":datasets.get_data("rsna"),
            #"Pneumothorax":datasets.get_data("siim"),
            }

NIH_Dataset num_samples=67310 views=['PA'] data_aug=None
NIH_Dataset num_samples=67310 views=['PA'] data_aug=None


In [16]:
perf = pd.DataFrame()
for ttarget in for_eval:
    for tmodel in models:
        print(ttarget,tmodel)
        if ttarget in tmodel.pathologies:
            dataset = for_eval[ttarget]
            limit = 2048
            auc = attribution.test_epoch(tmodel, for_eval[ttarget], ttarget, limit=limit)
            h = {}
            h["model"] = str(tmodel)
            h["target"] = ttarget
            h["auc"] = auc
            h["limit"] = limit
            
            perf = perf.append(h, ignore_index=True)
    

Cardiomegaly XRV-DenseNet121-all
0
10
Cardiomegaly XRV-DenseNet121-mimic_ch
0
10
Cardiomegaly jfhealthcare-DenseNet121
0
10
Mass XRV-DenseNet121-all
0
10
Mass XRV-DenseNet121-mimic_ch
Mass jfhealthcare-DenseNet121


In [17]:
perf

,auc,limit,model,target
0,0.907928,2048.0,XRV-DenseNet121-all,Cardiomegaly
1,0.697738,2048.0,XRV-DenseNet121-mimic_ch,Cardiomegaly
2,0.901575,2048.0,jfhealthcare-DenseNet121,Cardiomegaly
3,0.828224,2048.0,XRV-DenseNet121-all,Mass


In [18]:
pickle.dump(perf, open("perf.pkl", "bw"))

In [176]:
perf = pickle.load(open("perf.pkl", "br"))

In [19]:
perf.groupby(["target", "model"])[["auc"]].mean().unstack(level=1)

auc                           \
model        XRV-DenseNet121-all XRV-DenseNet121-mimic_ch   
target                                                      
Cardiomegaly            0.907928                 0.697738   
Mass                    0.828224                      NaN   

                                       
model        jfhealthcare-DenseNet121  
target                                 
Cardiomegaly                 0.901575  
Mass                              NaN

In [31]:
k = pd.DataFrame()
for ttarget in for_eval:
    for tmodel in models:
        print(ttarget,tmodel)
        if ttarget in tmodel.pathologies:
            dataset = for_eval[ttarget]
            a = attribution.run_eval(ttarget, dataset, tmodel, ae=ae, limit=5) #80)
            a["model"] = str(tmodel)
            k = k.append(a)
            #print(a)
    

Cardiomegaly XRV-DenseNet121-all
[0.6280689] -1000 390
[0.71163327] -880 120
[0.5574434] -810 220
[0.44826764] -370 20
[0.55092824] -830 460
[0.654263] -1000 260
Cardiomegaly XRV-DenseNet121-mimic_ch
[0.5761652] -1000 320
[0.7410898] -1000 1000
[0.6532508] -1000 1000
[0.20250566] -580 70
[0.37010485] -260 20
[0.55986613] -580 390
Cardiomegaly jfhealthcare-DenseNet121
[0.877294] -610 130
[0.62099916] -290 130
[0.4836145] -50 200
[0.47765264] -150 250
[0.5441582] -1 1
[0.5754823] -90 70
Mass XRV-DenseNet121-all
[0.28470787] -100 20
[0.6525935] -100 20
[0.51025075] -1000 1000
[0.502969] -1 1
[0.50576806] -660 520
[0.71170574] -620 110
Mass XRV-DenseNet121-mimic_ch
Mass jfhealthcare-DenseNet121


In [32]:
k

,distance_balanced,distance_95,inside_95,iou_95,inside_balanced,iou_balanced,precision_95,recall_95,precision_balanced,recall_balanced,idx,p,target,method,model
0,91.403458,88.581109,0.007225,0.005642,0.111468,0.059024,0.025110,0.007225,0.111468,0.111468,980,0.786204,Cardiomegaly,latentshift-max,XRV-DenseNet121-all
1,66.372503,61.939622,0.095251,0.084325,0.271864,0.157316,0.423675,0.095251,0.271864,0.271864,1686,0.882725,Cardiomegaly,latentshift-max,XRV-DenseNet121-all
2,74.373083,55.999430,0.116615,0.107268,0.385171,0.238521,0.572340,0.116615,0.385171,0.385171,1918,0.708709,Cardiomegaly,latentshift-max,XRV-DenseNet121-all
3,64.764706,57.715176,0.139328,0.122632,0.312912,0.185474,0.505779,0.139328,0.312912,0.312912,2121,0.509952,Cardiomegaly,latentshift-max,XRV-DenseNet121-all
4,71.278294,68.792009,0.090122,0.077911,0.296438,0.174011,0.365086,0.090122,0.296438,0.296438,3356,0.626276,Cardiomegaly,latentshift-max,XRV-DenseNet121-all
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,53.379721,53.192072,0.487603,0.217645,0.278926,0.162065,0.282184,0.487603,0.278926,0.278926,2626,0.852337,Mass,guided,XRV-DenseNet121-all
20,33.266145,50.748411,0.763636,0.033136,0.127273,0.067961,0.033479,0.763636,0.127273,0.127273,5167,0.608783,Mass,guided,XRV-DenseNet121-all
21,8.523983,23.311280,0.900800,0.218981,0.590400,0.418842,0.224392,0.900800,0.590400,0.590400,8710,0.601590,Mass,guided,XRV-DenseNet121-all
22,94.510696,77.123129,0.266667,0.018175,0.000000,0.000000,0.019131,0.266667,0.000000,0.000000,13254,0.502609,Mass,guided,XRV-DenseNet121-all


In [29]:
z1=np.zeros((150,200))
z2=np.random.rand(150,200)
z1[50:125,50:150]=1
z2[75:100,50:150]+=2
attribution.calc_iou(z1,z2)

{'inside_95': 0.0,
 'iou_95': 0.0,
 'inside': 0.25,
 'iou': 0.25,
 'precision': 1.0,
 'recall': 0.25}

In [180]:
import pickle
pickle.dump(k, open("ious80.pkl", "bw"))

In [181]:
k = pickle.load(open("ious80.pkl", "br"))

In [17]:
del k["idx"]

In [182]:
k.head()

,iou,precision,recall,idx,p,target,method,model
0,0.276051,0.432665,0.432665,980,0.786204,Cardiomegaly,latentshift-max,XRV-DenseNet121-all
1,0.352275,0.521011,0.521011,1686,0.882725,Cardiomegaly,latentshift-max,XRV-DenseNet121-all
2,0.424728,0.596223,0.596223,1918,0.708709,Cardiomegaly,latentshift-max,XRV-DenseNet121-all
3,0.265393,0.419463,0.419463,2121,0.509952,Cardiomegaly,latentshift-max,XRV-DenseNet121-all
4,0.367777,0.537773,0.537773,3356,0.626276,Cardiomegaly,latentshift-max,XRV-DenseNet121-all


In [35]:
kk = k.groupby(["target", "model","method"]).agg({'iou_balanced'    : ['mean', 'std'],
                                                  'iou_95' : ['mean', 'std'],
                                                  'inside_balanced' : ['mean', 'std'],
                                                  'inside_95' : ['mean', 'std'],
                                                  'distance_balanced' : ['mean', 'std'],
                                                  'distance_95' : ['mean', 'std']
                                                 }).round(2)
#kk

In [36]:
kk.head()

iou_balanced        \
                                                              mean   std   
target       model                    method                               
Cardiomegaly XRV-DenseNet121-all      grad                    0.35  0.03   
                                      guided                  0.29  0.04   
                                      integrated              0.30  0.07   
                                      latentshift-max         0.16  0.06   
             XRV-DenseNet121-mimic_ch grad                    0.25  0.09   

                                                      iou_95        \
                                                        mean   std   
target       model                    method                         
Cardiomegaly XRV-DenseNet121-all      grad              0.16  0.01   
                                      guided            0.13  0.03   
                                      integrated        0.13  0.06   
                                      latentshift-max   0.07  0.04   
             XRV-DenseNet121-mimic_ch grad              0.11  0.04   

                                                      inside_balanced        \
                                                                 mean   std   
target       model                    method                                  
Cardiomegaly XRV-DenseNet121-all      grad                       0.51  0.04   
                                      guided                     0.44  0.05   
                                      integrated                 0.46  0.08   
                                      latentshift-max            0.27  0.09   
             XRV-DenseNet121-mimic_ch grad                       0.39  0.12   

                                                      inside_95        \
                                                           mean   std   
target       model                    method                            
Cardiomegaly XRV-DenseNet121-all      grad                 0.17  0.01   
                                      guided               0.14  0.03   
                                      integrated           0.15  0.06   
                                      latentshift-max      0.08  0.05   
             XRV-DenseNet121-mimic_ch grad                 0.13  0.04   

                                                      distance_balanced  \
                                                                   mean   
target       model                    method                              
Cardiomegaly XRV-DenseNet121-all      grad                        53.53   
                                      guided                      53.89   
                                      integrated                  52.78   
                                      latentshift-max             74.26   
             XRV-DenseNet121-mimic_ch grad                        65.75   

                                                             distance_95  \
                                                         std        mean   
target       model                    method                               
Cardiomegaly XRV-DenseNet121-all      grad              3.23       47.63   
                                      guided            6.42       53.09   
                                      integrated       11.80       52.63   
                                      latentshift-max   9.64       68.52   
             XRV-DenseNet121-mimic_ch grad             15.91       61.44   

                                                              
                                                         std  
target       model                    method                  
Cardiomegaly XRV-DenseNet121-all      grad              2.59  
                                      guided            6.10  
                                      integrated       13.81  
                                      latentshift-max  12.73  
             XR

In [189]:
for c in kk.columns.get_level_values(0).unique():
    kk[c+"_"] =  kk[(c, "mean")].apply("{0:0.2f}".format) + "$\pm$" + kk[(c, "std")].apply("{0:0.2f}".format)
    del kk[c]
    kk[c] = kk[c+"_"]
    del kk[c+"_"]

In [190]:
kk.unstack(level=1)

iou                           \
                                                                            
model                        XRV-DenseNet121-all XRV-DenseNet121-mimic_ch   
target       method                                                         
Atelectasis  grad                  0.07$\pm$0.07            0.06$\pm$0.07   
             guided                0.09$\pm$0.08            0.04$\pm$0.04   
             integrated            0.05$\pm$0.05            0.04$\pm$0.05   
             latentshift-max       0.11$\pm$0.12            0.08$\pm$0.11   
Cardiomegaly grad                  0.35$\pm$0.05            0.25$\pm$0.09   
             guided                0.28$\pm$0.06            0.15$\pm$0.06   
             integrated            0.27$\pm$0.08            0.15$\pm$0.08   
             latentshift-max       0.33$\pm$0.07            0.21$\pm$0.09   
Effusion     grad                  0.12$\pm$0.09            0.08$\pm$0.08   
             guided                0.15$\pm$0.09            0.06$\pm$0.05   
             integrated            0.11$\pm$0.08            0.05$\pm$0.06   
             latentshift-max       0.16$\pm$0.11            0.11$\pm$0.11   
Lung Opacity grad                  0.21$\pm$0.11            0.13$\pm$0.09   
             guided                0.21$\pm$0.12            0.09$\pm$0.07   
             integrated            0.17$\pm$0.10            0.08$\pm$0.07   
             latentshift-max       0.20$\pm$0.13            0.15$\pm$0.14   
Mass         grad                  0.16$\pm$0.14                      NaN   
             guided                0.19$\pm$0.16                      NaN   
             integrated            0.13$\pm$0.13                      NaN   
             latentshift-max       0.14$\pm$0.17                      NaN   
Pneumothorax grad                  0.01$\pm$0.02            0.01$\pm$0.02   
             guided                0.03$\pm$0.05            0.02$\pm$0.03   
             integrated            0.01$\pm$0.02            0.01$\pm$0.01   
             latentshift-max       0.02$\pm$0.04            0.03$\pm$0.07   

                                                       
                                                       
model                        jfhealthcare-DenseNet121  
target       method                                    
Atelectasis  grad                       0.13$\pm$0.10  
             guided                     0.10$\pm$0.07  
             integrated                 0.10$\pm$0.09  
             latentshift-max            0.09$\pm$0.09  
Cardiomegaly grad                       0.45$\pm$0.04  
             guided                     0.31$\pm$0.05  
             integrated                 0.36$\pm$0.09  
             latentshift-max            0.35$\pm$0.09  
Effusion     grad                       0.18$\pm$0.10  
             guided                     0.14$\pm$0.07  
             integrated                 0.14$\pm$0.09  
             latentshift-max            0.16$\pm$0.10  
Lung Opacity grad                                 NaN  
             guided                               NaN  
             integrated                           NaN  
             latentshift-max                      NaN  
Mass         grad                                 NaN  
             guided                               NaN  
             integrated                           NaN  
             latentshift-max                      NaN  
Pneumothorax grad                                 NaN  
             guided                               NaN  
             integrated                           NaN  
             latentshift-max                      NaN

In [195]:
print(kk.unstack(level=1).to_latex(escape=False, ))

\begin{tabular}{lllll}
\toprule
             & {} & \multicolumn{3}{l}{iou} \\
             & {} \\
             & model & XRV-DenseNet121-all & XRV-DenseNet121-mimic_ch & jfhealthcare-DenseNet121 \\
target & method &                     &                          &                          \\
\midrule
Atelectasis & grad &       0.07$\pm$0.07 &            0.06$\pm$0.07 &            0.13$\pm$0.10 \\
             & guided &       0.09$\pm$0.08 &            0.04$\pm$0.04 &            0.10$\pm$0.07 \\
             & integrated &       0.05$\pm$0.05 &            0.04$\pm$0.05 &            0.10$\pm$0.09 \\
             & latentshift-max &       0.11$\pm$0.12 &            0.08$\pm$0.11 &            0.09$\pm$0.09 \\
Cardiomegaly & grad &       0.35$\pm$0.05 &            0.25$\pm$0.09 &            0.45$\pm$0.04 \\
             & guided &       0.28$\pm$0.06 &            0.15$\pm$0.06 &            0.31$\pm$0.05 \\
             & integrated &       0.27$\pm$0.08 &            0.15$\pm$0.08 &      